In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

%matplotlib inline

In [3]:
# For Presentation Populating
from pptx import Presentation
from pptx.util import Cm
from pptx.enum.text import MSO_ANCHOR, MSO_AUTO_SIZE
from pptx.dml.color import RGBColor
from pptx.enum.dml import MSO_THEME_COLOR
from pptx.util import Pt

In [5]:
pd.options.display.max_rows = 100

### Active Functions

In [6]:
def cleaner(df,start):
    # This function aims to tokenise the set, filter out stopwords/punctuations ,etc. and translate any remaining words not captured by earlier machine translations. 
    # Will be assigned back to individual cell locations after
    
    # Create a temporary dict within the function to work on
    tempdf = df.copy()

    # lowercase each cell's words. Token out into individual words, returning the list container to the cell. Only if the cell is a str. Otherwise return 0 to the cell.
    tempdf.iloc[:,start:]= tempdf.iloc[:,start:].map(lambda x: nltk.word_tokenize(x.lower()) if isinstance(x, str) else 0)

    # Filter out stopwords if cell is a list (due to tokenisation step) else return unedited cell value
    # stop_words is a global set containing the words to filter. Has additional items such as punctuation from the base stopwords form
    tempdf.iloc[:,start:] = tempdf.iloc[:,start:].map(lambda x: [w for w in x if not w in stop_words]
                                         if isinstance(x, list) else x)

    # Search if cell word is in dict of translated word. Only if cell is a list (due to tokenisation step) otherwise return unedited cell value.
    # translated_word is a global dict containing the word as keys and translations of the words as values
    #tempdf.iloc[:,start:] = tempdf.iloc[:,start:].map(lambda x: [translated_words[w] if w in translated_words.keys() else w for w in x]
                                          #if isinstance(x, list) else x)
      
    return tempdf

In [8]:
# generating list from nested list
def wordlistnestgen(df, cols):
    
    # Create a temporary dict within the function to work on
    tempdf = df.copy()
    tempdf = tempdf.iloc[:,cols]

    # Create a full list of all words in all cells
    list_words = [item for item in tempdf.map(lambda x: x if isinstance(x, list) else None).to_numpy().flatten() if item is not None]

    return list_words

In [10]:
# Creating word strings for cloud from list
def stringgetter(df):

    #flatten df to list
    flat_list = [
        x
        for xs in df
        for x in xs]

    # strip of whitespaces
    for x in flat_list:
        x = x.strip()

    # make to stroig
    s = " ".join(flat_list)

    return s

In [12]:
# generate df to check the words
def wordlister(df, start, cols = list(range(5,36))):
    tempdf = cleaner(df,5)
    listwords = wordlistnestgen(tempdf, cols)

    flat_list = [
        x
        for xs in listwords
        for x in xs]
    
    dflist = pd.Series(flat_list)
    counts = dflist.value_counts()
    dfcounts = counts.reset_index()

    return dfcounts

In [14]:
# Generate text to copy-paste and the table of text in the cloud
def textcheck(df, start, cols):

    listwords = wordlister(df, start, cols)
    
    #print top 5 words in the format indicated
    strlist = ''
    for i, r in listwords.iloc[0:5].iterrows():
        strlist += f'{r['index'].replace('_', ' ').capitalize()} | '

    # return list for access later
    return listwords, strlist

In [16]:
def cloudgenerator(df, start, cols, hsl, filename = 'test', size = 50, width = 1600, height = 800, dirname = 'test', wordlist = 'NA'):  
    tempdf = cleaner(df, start)
    listwords = wordlistnestgen(tempdf, cols)
    cloudstring = stringgetter(listwords)

    # tuple unpack hsl values
    hue, sat, lumibright, lumirand = hsl
    if len(lumirand) == 2:
        hue2, sat2 = hue, sat
        lumil, lumiu = lumirand
    else:
        hue2, sat2, lumil, lumiu = lumirand
    
    
    # Set custom colour. The other args are currently unused but could be utilised to affect the color scheme. e.g. hue = len(word) * basehue to use word length to affect hue
    def color_func(word, font_size, position,orientation,random_state=None, **kwargs):
        if word in wordlist:
            return(f"hsl({hue}, {sat}%, {lumibright}%)")
        else:
            return(f"hsl({hue2}, {sat2}%, {np.random.randint(lumil , lumiu)}%)")
        
    wordcloud = WordCloud(font_path = 'Lato-Regular.ttf',width=width, height=height, collocations=False, max_words = size, background_color = 'white', normalize_plurals = False, relative_scaling = 0.3).generate(cloudstring)

    # Display the generated image:
    plt.figure(figsize=(20,10))
    plt.imshow(wordcloud.recolor(color_func = color_func), interpolation='bilinear')
    plt.axis("off")
    plt.savefig(f'Clouds/{dirname}/{filename}.png',bbox_inches='tight')
    plt.close()

In [18]:
def cloudplacer(slidekey, wordcloud, positionkey):
    slide = prs.slides.get(slidekey)  #Get the slide
    if '_all' in positionkey:
        pic = slide.shapes.add_picture(wordcloud,left=Cm(2.59),top=Cm(2.03),width=Cm(20.22),height=Cm(10.24))
    elif '_sweet' in positionkey:
        pic = slide.shapes.add_picture(wordcloud,left=Cm(1.93),top=Cm(2.03),width=Cm(20.22),height=Cm(10.24))
    elif '_UA' in positionkey:
        pic = slide.shapes.add_picture(wordcloud,left=Cm(1.12),top=Cm(5.3),width=Cm(6.17),height=Cm(6.78))
    elif '_dressing' in positionkey:
        pic = slide.shapes.add_picture(wordcloud,left=Cm(9.62),top=Cm(5.3),width=Cm(6.17),height=Cm(6.78))
    else:
        pic = slide.shapes.add_picture(wordcloud,left=Cm(18.12),top=Cm(5.3),width=Cm(6.17),height=Cm(6.78))

In [20]:
# For the text on the 'Overall' Slide
def textplacer(slidekey, text, boxposition, color):

    slide = prs.slides.get(slidekey)
    left, top, width, height, bold = boxposition
    R, G, B = color
    
    shape = slide.shapes.add_textbox(Cm(left),Cm(top),Cm(width),Cm(height))
    text_frame = shape.text_frame
    text_frame.margin_top = Cm(0.13)
    text_frame.margin_bottom = Cm(0.13)
    text_frame.margin_left = Cm(0.25)
    text_frame.vertical_anchor = MSO_ANCHOR.TOP
    text_frame.word_wrap = True
    text_frame.auto_size = MSO_AUTO_SIZE.SHAPE_TO_FIT_TEXT
    
    p = text_frame.add_paragraph()
    
    run1 = p.add_run()
    run1.text = 'Top 5 Flavours: '
    run1.font.name = 'Lato Light'
    run1.font.size = Pt(10)
    run1.font.bold = True
    run1.font.color.rgb = RGBColor(33, 33, 33)
    
    run2 = p.add_run()
    run2.text = text
    run2.font.name = 'Lato Light'
    run2.font.size = Pt(10)
    run2.font.bold = bold
    run2.font.color.rgb = RGBColor(R, G, B) # TO EDIT

## Code

### Creating the different df for global, regions and country

In [22]:
df = pd.read_excel(r"Translated.xlsx", sheet_name = "A1")

In [23]:
GLOBALdf = df.copy()
APACdf = df[df['Regions'] == 'APAC'].copy()
EUROPEdf = df[df['Regions'] == 'EUROPE'].copy()
LATAMdf = df[df['Regions'] == 'LATAM'].copy()
CHINAdf = df[df['hCountry'] == 1].copy()
INDONESIAdf = df[df['hCountry'] == 2].copy()
INDIAdf = df[df['hCountry'] == 3].copy()
UKdf = df[df['hCountry'] == 4].copy()
GERMANYdf = df[df['hCountry'] == 5].copy()
FRANCEdf = df[df['hCountry'] == 6].copy()
BRAZILdf = df[df['hCountry'] == 7].copy()
MEXICOdf = df[df['hCountry'] == 8].copy()
USdf = df[df['hCountry'] == 9].copy()
MOROCCOdf = df[df['hCountry'] == 10].copy()

In [26]:
dfdict = {'GLOBAL':GLOBALdf
          ,'APAC':APACdf
          ,'EUROPE':EUROPEdf
          ,'LATAM':LATAMdf
          ,'CHINA':CHINAdf
          ,'INDONESIA':INDONESIAdf
          ,'INDIA':INDIAdf
          ,'UK':UKdf
          ,'GERMANY':GERMANYdf
          ,'FRANCE':FRANCEdf
          ,'BRAZIL':BRAZILdf
          ,'MEXICO':MEXICOdf
          ,'US':USdf
          ,'MOROCCO':MOROCCOdf
}

### Directions

In [28]:
# set stopwords
stop_words = set(stopwords.words('english'))
addstopwords = [' ,', ',',"n't",'remember', '.',' .', 'know', 'much', 'sensation', 'gives', 'color', 'colour','💋','/','-','🥗','🍓','🍔','😘','🍡','🍬', 'seem','seems','taste','tastes','food','powder','look','less','bit','little','something','somewhat','lot','maybe','rather','feel','feels',
               'may','made', 'sense','must', 'like', 'little', 'more', 'sauce', 'se', 'le']
stop_words.update(addstopwords)

In [32]:
# dict to store generation instructions
# order is {colour_scope: (columns, hsl(hue, sat, lumi luminrand(hue2 (optional), sat2(optional), lumi_lower, Lumi_upper), max_words, width, height, colorNumberCode)}
directions = {# PINK
             '_pink_all':([5,14,23], (340, 100, 59.8, (78, 80)), 50, 1600, 800, 1),
             '_pink_UA':([5], (340, 100, 59.8, (78, 80)), 25, 1000, 1100, 1),
             '_pink_dressing':([14], (340, 100, 59.8, (78, 80)), 25, 1000, 1100, 1),
             '_pink_season':([23], (340, 100, 59.8, (78, 80)), 25, 1000, 1100, 1),
             # YELLOW
             '_yellow_all':([6,15,24], (45, 100, 50, (80, 90)), 50, 1600, 800, 2),
             '_yellow_UA':([6], (45, 100, 50, (80, 90)), 25, 1000, 1100, 2),
             '_yellow_dressing':([15], (45, 100, 50, (80, 90)), 25, 1000, 1100, 2),
             '_yellow_season':([24], (45, 100, 50, (80, 90)), 25, 1000, 1100, 2),
             # BLUE
             '_blue_all':([7,16,25], (235, 100, 50, (80, 90)), 50, 1600, 800, 3),
             '_blue_UA':([7], (235, 100, 50, (80, 90)), 25, 1000, 1100, 3),
             '_blue_dressing':([16], (235, 100, 50, (80, 90)), 25, 1000, 1100, 3),
             '_blue_season':([25], (235, 100, 50, (80, 90)), 25, 1000, 1100, 3),
             # PURPLE
             '_purple_all':([8,17,26], (264, 31.4, 52, (264, 31.1, 76, 77)), 50, 1600, 800, 4),
             '_purple_UA':([8], (264, 31.4, 52, (264, 31.1, 76, 77)), 25, 1000, 1100, 4),
             '_purple_dressing':([17], (264, 31.4, 52, (264, 31.1, 76, 77)), 25, 1000, 1100, 4),
             '_purple_season':([26], (264, 31.4, 52, (264, 31.1, 76, 77)), 25, 1000, 1100, 4),
             # ORANGE
             '_orange_all':([9,18,27], (30, 100, 50, (80, 90)), 50, 1600, 800, 5),
             '_orange_UA':([9], (30, 100, 50, (80, 90)), 25, 1000, 1100, 5),
             '_orange_dressing':([18], (30, 100, 50, (80, 90)), 25, 1000, 1100, 5),
             '_orange_season':([27], (30, 100, 50, (80, 90)), 25, 1000, 1100, 5),
             # BROWN
             '_brown_all':([10,19,28], (30, 100, 25, (80, 90)), 50, 1600, 800, 6),
             '_brown_UA':([10], (30, 100, 25, (80, 90)), 25, 1000, 1100, 6),
             '_brown_dressing':([19], (30, 100, 25, (80, 90)), 25, 1000, 1100,6),
             '_brown_season':([28], (30, 100, 25, (80, 90)), 25, 1000, 1100, 6),
             # BLACK
             '_black_all':([11,20,29],(0, 0, 0, (60, 70)), 50, 1600, 800, 7),
             '_black_UA':([11],(0, 0, 0, (60, 70)), 25, 1000, 1100, 7),
             '_black_dressing':([20], (0, 0, 0, (60, 70)), 25, 1000, 1100, 7),
             '_black_season':([29],(0, 0, 0, (60, 70)), 25, 1000, 1100, 7),
             # RED
             '_red_all':([12,21,30],(0, 100, 50, (70, 80)), 50, 1600, 800, 8),
             '_red_UA':([12],(0, 100, 50, (70, 80)), 25, 1000, 1100, 8),
             '_red_dressing':([21],(0, 100, 50, (70, 80)), 25, 1000, 1100, 8),
             '_red_season':([30],(0, 100, 50, (70, 80)), 25, 1000, 1100, 8),
             # GREEN
             '_green_all':([13,22,31],(143, 76.5, 33.3, (120, 38.2, 73, 74)), 50, 1600, 800, 9),
             '_green_UA':([13],(143, 76.5, 33.3, (120, 38.2, 73, 74)), 25, 1000, 1100, 9),
             '_green_dressing':([22],(143, 76.5, 33.3, (120, 38.2, 73, 74)), 25, 1000, 1100, 9),
             '_green_season':([31],(143, 76.5, 33.3, (120, 38.2, 73, 74)), 25, 1000, 1100, 9),
             # SWEET CATEGORIES
             '_blue_sweet':([32], (235, 100, 50, (80, 90)), 50, 1600, 800, 3),
             '_pink_sweet':([33], (340, 100, 59.8, (78, 80)), 50, 1600, 800, 1),
             '_orange_sweet':([34], (30, 100, 50, (80, 90)), 50, 1600, 800, 5),
             '_red_sweet':([35],(0, 100, 50, (70, 80)), 50, 1600, 800, 8)}

In [34]:
pptdirection = {2:['_pink_all'],
             3:['_pink_UA', '_pink_dressing', '_pink_season'],
             # YELLOW
             4:['_yellow_all'],
             5:['_yellow_UA', '_yellow_dressing', '_yellow_season'],
             # BLUE
             6:['_blue_all'],
             7:['_blue_UA', '_blue_dressing', '_blue_season'],
             # PURPLE
             8:['_purple_all'],
             9:['_purple_UA', '_purple_dressing', '_purple_season'],
             # ORANGE
             10:['_orange_all'],
             11:['_orange_UA', '_orange_dressing', '_orange_season'],
             # BROWN
             12:['_brown_all'],
             13:['_brown_UA', '_brown_dressing', '_brown_season'],
             # BLACK
             14:['_black_all'],
             15:['_black_UA', '_black_dressing', '_black_season'],
             # RED
             16:['_red_all'],
             17:['_red_UA', '_red_dressing', '_red_season'],
             # GREEN
             18:['_green_all'],
             19:['_green_UA', '_green_dressing', '_green_season'],
             # SWEET CATEGORIES
             22:['_blue_sweet'],
             21:['_pink_sweet'],
             23:['_orange_sweet'],
             24:['_red_sweet']
               }

colordirections = {'pink': (255,102,204),
                   'yellow': (199, 149, 0),
                   'blue': (3, 104, 156),
                   'purple':(54, 0, 163),
                   'orange':(237, 129, 22),
                   'brown':(112, 56, 0),
                   'black':(33, 33, 33),
                   'red':(193, 30, 50),
                   'green':(83, 129, 70)
                  }

boxdirections = {'all':(0.5, 12.27, 16.87, 1.37, True),
                 'sweet':(0.5, 12.27, 16.87, 1.37, True),
                 'UA':(0.03, 12.27, 8.33, 1.37, False),
                 'dressing':(8.53, 12.27, 8.33, 1.37, False),
                 'season':(17.03, 12.27, 8.33, 1.37, False)
                }

## Generating word cloud and Populating the PPTs

In [60]:
# Main Programme
startcol = 5

# dictionary to store the listnames for access later if want to check
var_names = {}
ppttext = {}

for name, df in dfdict.items():
    for k,v in directions.items():
        # print copy-paste text and store list for access later in needed
        tempdf, pptstring =  textcheck(df, start = startcol, cols = v[0])
        var_names[name+k] = tempdf
        ppttext[name+k] = pptstring
    
        # generate and save cloud as png
        cloudgenerator(df, start = startcol, cols = v[0] , hsl = v[1], filename = name+k, size = v[2], width = v[3], height = v[4], dirname = name, wordlist = (tempdf.iloc[0:5,0]).tolist())


    prs = Presentation(rf'Submissions\Food Colouring_WordCloud_{name}.pptx')
    
    page_id = {i+1 : slide.slide_id for i, slide in enumerate(prs.slides)}

    # First, go through the slides
    for k, v in page_id.items():
        # If slide index is registered as needing an image, go in
        if k in pptdirection.keys():
            # Iterate through the value (list) to place images one by one
            for i in pptdirection[k]:
                # Find and store the path of the wordcloud              
                wordcloud = rf'CLouds\{name}\{name+i}.png'
                text = ppttext[name+i]
                #place the wordcloud
                cloudplacer(v, wordcloud, i)
                #place the text
                pos = i.split('_')
                textplacer(v, ppttext[name+i], boxdirections[pos[2]], colordirections[pos[1]])
        else:
            continue

    prs.save(rf'Submissions\Food Colouring_WordCloud_{name}.pptx')

In [54]:
dfdicttest = {'GLOBAL':GLOBALdf
           ,'APAC':APACdf
          ,'EUROPE':EUROPEdf
          ,'LATAM':LATAMdf
          ,'CHINA':CHINAdf
          ,'INDONESIA':INDONESIAdf
          ,'INDIA':INDIAdf
          ,'UK':UKdf
          ,'GERMANY':GERMANYdf
          ,'FRANCE':FRANCEdf
          ,'BRAZIL':BRAZILdf
          ,'MEXICO':MEXICOdf
          ,'US':USdf
          ,'MOROCCO':MOROCCOdf
}

In [56]:
dfdictnumberstest = {'GLOBAL':1110
           ,'APAC':334
          ,'EUROPE':443
          ,'LATAM':222
          ,'CHINA':112
          ,'INDONESIA':111
          ,'INDIA':111
          ,'UK':111
          ,'GERMANY':111
          ,'FRANCE':113
          ,'BRAZIL':111
          ,'MEXICO':111
          ,'US':111
          ,'MOROCCO':108
}

In [47]:
startcol = 5
percentdf = {}

with pd.ExcelWriter(r'testpercentages.xlsx', engine="xlsxwriter") as writer:
    
    workbook=writer.book
    
    for regionname, df in dfdicttest.items():
        
        excelcol = 0
        
        worksheet=workbook.add_worksheet(regionname)
        writer.sheets[regionname] = worksheet
        
        for k,v in directions.items():
    
            tempdf, pptstring =  textcheck(df, start = startcol, cols = v[0])
            tempdf['Percentage'] = round((tempdf['count'] / (dfdictnumberstest[regionname]*len(v[0]))*100), 2)
            tempdf['Base Size'] = dfdictnumberstest[regionname]*len(v[0])
            tempdf.rename(columns = {'index':k}, inplace = True)
            tempdf.set_index(k, drop = True, inplace = True)

            tempdf.to_excel(writer, sheet_name = regionname, startrow=0 , startcol = excelcol) # change start col dynamically
            
            excelcol += 5

In [278]:
var_names['GLOBAL_pink_all']

,index,count
0,sweet,387
1,strawberry,246
2,salty,130
3,meat,113
4,fresh,111
...,...,...
565,unbalanced,1
566,fun,1
567,chilli_flakes,1
568,glaze,1


## Tester Codes

In [ ]:
# code to access list. Change the key as needed
var_names['global_pink_all'].head(100)

In [45]:
#tester prints out indiviual clouds as needed
mainname = 'test'
dftouse = APACdf
category = '_brown_all'

# load in values. change the dictionary key
cols, hsl, max_size, width, height, colorcode = directions[category]

# print copy-paste text for access

testdf2 , teststringholder = textcheck(dftouse, 5, cols = cols)
testholder = testdf2
print(teststringholder)

# generate and save cloud as png
cloudgenerator(dftouse, 5, cols, hsl, filename = f'{mainname+category}', size = max_size, width = width, height = height, dirname = mainname,  wordlist = (testdf2.iloc[0:5,0]).tolist())

Salty | Sweet | Strong | Barbeque | Chocolate | 


In [36]:
def cloudgeneratorcustom(df, start, cols, hsl, filename = 'test', size = 50, width = 1600, height = 800, dirname = 'test', wordlist = 'NA'):  
    tempdf = cleaner(df, start)
    listwords = wordlistnestgen(tempdf, cols)
    cloudstring = stringgetter(listwords)

    # tuple unpack hsl values
    hue, sat, lumibright, lumirand = hsl
    if len(lumirand) == 2:
        hue2, sat2 = hue, sat
        lumil, lumiu = lumirand
    else:
        hue2, sat2, lumil, lumiu = lumirand
    
    
    # Set custom colour. The other args are currently unused but could be utilised to affect the color scheme. e.g. hue = len(word) * basehue to use word length to affect hue
    def color_func(word, font_size, position,orientation,random_state=None, **kwargs):
        if word in wordlist:
            return(f"hsl({hue}, {sat}%, {lumibright}%)")
        else:
            return(f"hsl({hue2}, {sat2}%, {np.random.randint(lumil , lumiu)}%)")
        
    wordcloud = WordCloud(font_path = 'Lato-Regular.ttf',width=width, height=height, collocations=False, max_words = size, background_color = 'white', normalize_plurals = False, relative_scaling = 0.3, max_font_size = 1600).generate(cloudstring)

    # Display the generated image:
    plt.figure(figsize=(20,10))
    plt.imshow(wordcloud.recolor(color_func = color_func), interpolation='bilinear')
    plt.axis("off")
    plt.savefig(f'Clouds/{dirname}/{filename}.png',bbox_inches='tight')
    plt.close()

In [66]:
#prints out indiviual clouds as needed from cutom generator
mainname = 'MOROCCO'
dftouse = MOROCCOdf
category = '_orange_sweet'


# load in values. change the dictionary key
cols, hsl, max_size, width, height, colorcode = directions[category]

# print copy-paste text for access

testdf2 , teststringholder = textcheck(dftouse, 5, cols = cols)
testholdercustom = testdf2
print(teststringholder)

# generate and save cloud as png
cloudgeneratorcustom(dftouse, 5, cols, hsl, filename = f'{mainname+category}', size = max_size, width = width, height = height, dirname = mainname,  wordlist = (testdf2.iloc[0:5,0]).tolist())

Orange | Lemon | Mango | Sweet | Caramel | 


In [84]:
# For pink sweet cloud generation with new color scheme
for name, df in dfdict.items():

    mainname = name
    dftouse = df
    category = '_pink_sweet'
    
    # load in values. change the dictionary key
    cols, hsl, max_size, width, height, colorcode = directions[category]
    
    # print copy-paste text for access
    
    testdf2 , teststringholder = textcheck(dftouse, 5, cols = cols)
    testholdercustom = testdf2
    print(teststringholder)
    
    # generate and save cloud as png
    cloudgenerator(dftouse, 5, cols, hsl, filename = f'{mainname+category}', size = max_size, width = width, height = height, dirname = mainname,  wordlist = (testdf2.iloc[0:5,0]).tolist())

Strawberry | Sweet | Raspberry | Sugar | Cherry | 
Sweet | Strawberry | Ice cream | Fresh | Cherry | 
Strawberry | Sweet | Raspberry | Sugar | Berry | 
Strawberry | Sweet | Cherry | Candy | Gum | 
Strawberry | Sweet | Peach | Cherry | Ice cream | 
Sweet | Strawberry | Fresh | Delicious | Soft | 
Strawberry | Sweet | Ice cream | Rose | Tasty | 
Strawberry | Sweet | Raspberry | Cotton candy | Candy | 
Sweet | Strawberry | Raspberry | Fruity | Berry | 
Strawberry | Sugar | Sweet | Raspberry | Cookie | 
Strawberry | Sweet | Gum | Tutti frutti | Tasty | 
Strawberry | Sweet | Cherry | Candy | Sugar | 
Sweet | Strawberry | Cotton candy | Raspberry | Bubblegum | 
Strawberry | Sweet | Berry | Pomegranate | Sugar | 


In [78]:
testholdercustom.head(100)

,index,count
0,sweet,46
1,tutti_frutti,18
2,blueberry,18
3,ice_cream,12
4,cotton_candy,12
5,gum,11
6,refreshing,9
7,blue_sky,9
8,artificial,9
9,soft,7


In [246]:
#prints out indiviual clouds as needed from base generator
mainname = 'GERMANY'
dftouse =GERMANYdf
category = '_pink_sweet'

# load in values. change the dictionary key
cols, hsl, max_size, width, height, colorcode = directions[category]

# print copy-paste text for access

testdf2 , teststringholder = textcheck(dftouse, 5, cols = cols)
testholder = testdf2
print(teststringholder)

# generate and save cloud as png
cloudgenerator(dftouse, 5, cols, hsl, filename = f'{mainname+category}', size = max_size, width = width, height = height, dirname = mainname,  wordlist = (testdf2.iloc[0:5,0]).tolist())

Sweet | Strawberry | Raspberry | Fruity | Berry | 


In [88]:
testholder.sort_values(['count','index'],ascending = [False, True]).reset_index().head(100)

,level_0,index,count
0,0,sweet,46
1,2,blueberry,18
2,1,tutti_frutti,18
3,4,cotton_candy,12
4,3,ice_cream,12
5,5,gum,11
6,8,artificial,9
7,7,blue_sky,9
8,6,refreshing,9
9,10,candy,7


In [92]:
testdf2

,index,count
0,black_pepper,62
1,pepper,26
2,strong,17
3,bitter,16
4,tasty,14
...,...,...
140,chilli_powder,1
141,earthly,1
142,fungs,1
143,toasted,1


## Archive

def replacer(df, start):
    # This function aims to go through the df and replace detected 2-word words with its underscored variant
    
    # Create a temporary dict within the function to work on
    tempdf = df.copy()

    # Lowercase the df
    tempdf.iloc[:,start:] = tempdf.iloc[:,start:].map(lambda x: x.lower() if isinstance(x, str) else 0)

    # Replace words
    tempdf.iloc[:,start:] = tempdf.iloc[:,start:].replace(joinwords, regex = True)

    return tempdf

In [123]:
# Generate text to copy-paste and the table of text in the cloud
def textcheck(df, start, cols, colorcode, headname):

    listwords = wordlister(df, start, cols)
    
    # total = ((df['hP16'] == colorcode).sum())*len(cols)

    #print top 5 words in the format indicated
    print(f'{headname}', end = ' = ')
    for i, r in listwords.iloc[0:5].iterrows():
        print(f'{r['index'].capitalize()}', end = ' | ')

    # Newline for next set of top-5 words
    print('\n')

    # return list for access later
    return listwords

In [91]:
def cloudgenerator(df, start, cols, hsl, filename = 'test', size = 50, width = 1600, height = 800, dirname = 'test', wordlist = 'NA'):  
    tempdf = cleaner(df, start)
    listwords = wordlistnestgen(tempdf, cols)
    cloudstring = stringgetter(listwords)

    # tuple unpack hsl values
    hue, sat, lumil, lumiu = hsl
    
    # Set custom colour. The other args are currently unused but could be utilised to affect the color scheme. e.g. hue = len(word) * basehue to use word length to affect hue
    def color_func(word, font_size, position,orientation,random_state=None, **kwargs):
        if word in wordlist:
            return(f"hsl({hue}, {sat}%, {np.random.randint(lumil, lumiu)}%)")
        else:
            return(f"hsl({hue}, {sat}%, {np.random.randint(lumil, lumiu)}%)")
        
    wordcloud = WordCloud(font_path = 'Lato-Regular.ttf',width=width, height=height, collocations=False, max_words = size, background_color = 'white').generate(cloudstring)

    # Display the generated image:
    plt.figure(figsize=(20,10))
    plt.imshow(wordcloud.recolor(color_func = color_func), interpolation='bilinear')
    plt.axis("off")
    plt.savefig(f'Clouds/{dirname}/{filename}.png',bbox_inches='tight')
    plt.close()

In [220]:
for name in dfdict.keys():
    prs = Presentation(rf'Submissions\Food Colouring_WordCloud_{name}.pptx')
    
    page_id = {i+1 : slide.slide_id for i, slide in enumerate(prs.slides)}

    # First, go through the slides
    for k, v in page_id.items():
        # If slide index is registered as needing an image, go in
        if k in pptdirection.keys():
            # Iterate through the value (list) to place images one by one
            for i in pptdirection[k]:
                # Find and store the path of the wordcloud              
                wordcloud = rf'CLouds\{name}\{name+i}.png'
                text = ppttext[name+i]
                #place the wordcloud
                cloudplacer(v, wordcloud, i)
                #place the text
                pos = i.split('_')
                textplacer(v, ppttext[name+i], boxdirections[pos[2]], colordirections[pos[1]])
        else:
            continue

    prs.save(rf'Submissions\Food Colouring_WordCloud_{name}.pptx')

## Emotions

In [94]:
emotionsdf = pd.read_excel(r'emotions_colours.xlsx', sheet_name = 'GLOBAL', skiprows = 10)

In [96]:
emotionsdf.head()

,Unnamed: 0,Global,APAC,Europe,LATAM,America,China,Indonesia,India,United Kingdom,Germany,France,Brazil,Mexico,United States,Morocco
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Unweighted Base,1110,334,443,222,111,112,111,111,111,111,113,111,111,111,108
2,Weighted Total,1111,333,444,222,111,111,111,111,111,111,111,111,111,111,111
3,Mystery (Net),0.1546,0.205,0.1185,0.1501,0.1572,0.3135,0.163,0.1385,0.1095,0.071,0.1104,0.1689,0.1313,0.1572,0.1831
4,Elegance,0.1054,0.1379,0.0949,0.0912,0.0778,0.1555,0.1432,0.1151,0.0705,0.0494,0.0852,0.1018,0.0806,0.0778,0.1744


In [98]:
emotionsdf.dropna(inplace = True)

In [99]:
emotionsdf.reset_index(inplace = True)
emotionsdf.drop('index', axis = 1, inplace = True)

In [102]:
emotionsdf

,Unnamed: 0,Global,APAC,Europe,LATAM,America,China,Indonesia,India,United Kingdom,Germany,France,Brazil,Mexico,United States,Morocco
0,Unweighted Base,1110,334,443,222,111,112,111,111,111,111,113,111,111,111,108
1,Weighted Total,1111,333,444,222,111,111,111,111,111,111,111,111,111,111,111
2,Mystery (Net),0.1546,0.205,0.1185,0.1501,0.1572,0.3135,0.163,0.1385,0.1095,0.071,0.1104,0.1689,0.1313,0.1572,0.1831
3,Elegance,0.1054,0.1379,0.0949,0.0912,0.0778,0.1555,0.1432,0.1151,0.0705,0.0494,0.0852,0.1018,0.0806,0.0778,0.1744
4,Sophistication,0.0577,0.0847,0.0292,0.0589,0.0882,0.2036,0.0198,0.0307,0.0444,0.0305,0.0252,0.0671,0.0507,0.0882,0.0166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
508,Happiness,0.0743,0.0986,0.0642,0.0482,0.0938,0.074,0.113,0.1088,0.0438,0.0649,0.0496,0.0418,0.0546,0.0938,0.0984
509,Joy,0.0582,0.0629,0.0511,0.0489,0.0911,0.0445,0.0256,0.1186,0.0441,0.0872,0.0243,0.0469,0.051,0.0911,0.0488
510,Positivity,0.1097,0.1153,0.1167,0.1076,0.0695,0.1094,0.0749,0.1616,0.0514,0.1688,0.1166,0.1152,0.1,0.0695,0.1299
511,Fun,0.0576,0.0617,0.0496,0.0647,0.0628,0.0797,0.022,0.0835,0.0736,0.0607,0.0464,0.0484,0.081,0.0628,0.0178


In [103]:
# color keys
colororder = ['pink','yellow','blue','purple','orange','brown','black','red','green']

colordict = {}

# Iterate through excel and extract individual color dfs
counter = 0
for i in range(57,514,57):
    j = i-57
    colordict[colororder[counter]] = emotionsdf.iloc[j:i].copy()
    counter += 1

In [106]:
for k, v in colordict.items():
    v.reset_index(drop = True, inplace = True)
    v.drop([0,1,2,5,10,17,24,32,38,44,50,56], inplace = True)
    v.reset_index(drop = True, inplace = True)
    v.rename(columns = {'Unnamed: 0':'EMOTIONS','United Kingdom':'UK','United States':'US'}, inplace = True)
    v.rename(columns=str.upper, inplace = True)
    v.drop(['AMERICA'], axis = 1, inplace = True)
    colordict[k] = v.map(lambda x : round(x*10000) if isinstance(x, float) else x)

In [108]:
colordict['pink']

,EMOTIONS,GLOBAL,APAC,EUROPE,LATAM,CHINA,INDONESIA,INDIA,UK,GERMANY,FRANCE,BRAZIL,MEXICO,US,MOROCCO
0,Elegance,1054,1379,949,912,1555,1432,1151,705,494,852,1018,806,778,1744
1,Sophistication,577,847,292,589,2036,198,307,444,305,252,671,507,882,166
2,Confidence,556,437,636,487,506,586,219,967,866,454,386,588,726,258
3,Power,304,329,227,241,348,134,504,246,117,83,396,85,667,460
4,Success,342,364,411,183,240,433,418,299,447,425,288,77,319,473
5,Trust,548,776,532,345,518,1017,793,117,607,953,321,368,334,450
6,Comfort,854,982,955,577,1027,441,1477,742,534,1147,774,380,625,1396
7,Naturality,928,932,1022,830,440,1115,1240,745,1128,1021,652,1008,732,1196
8,Organic,392,592,226,406,351,335,1091,370,276,166,483,329,428,92
9,Warmth,1065,890,1167,1105,805,1206,658,1875,1126,182,679,1531,1101,1484


In [110]:
emotionsdirections = {# PINK
             'pink':((340, 100, 59.8, (78, 80)), 50, 1600, 800, 1),
             # YELLOW
             'yellow':((45, 100, 50, (80, 90)), 50, 1600, 800, 2),
             # BLUE
             'blue':((235, 100, 50, (80, 90)), 50, 1600, 800, 3),
             # PURPLE
             'purple':((264, 31.4, 52, (264, 31.1, 76, 77)), 50, 1600, 800, 4),
             # ORANGE
             'orange':((30, 100, 50, (80, 90)), 50, 1600, 800, 5),
             # BROWN
             'brown':((30, 100, 25, (80, 90)), 50, 1600, 800, 6),
             # BLACK
             'black':((0, 0, 0, (60, 70)), 50, 1600, 800, 7),
             # RED
             'red':((0, 100, 50, (70, 80)), 50, 1600, 800, 8),
             # GREEN
             'green':((143, 76.5, 33.3, (120, 38.2, 73, 74)), 50, 1600, 800, 9),
}

regiondirections = {'GLOBAL':1
          ,'APAC':2
          ,'EUROPE':3
          ,'LATAM':4
          ,'CHINA':5
          ,'INDONESIA':6
          ,'INDIA':7
          ,'UK':8
          ,'GERMANY':9
          ,'FRANCE':10
          ,'BRAZIL':11
          ,'MEXICO':12
          ,'US':13
          ,'MOROCCO':14
}

In [112]:
def cloudgeneratoremotion(freqdict, hsl, filename = 'test', size = 50, width = 1600, height = 800, dirname = 'test', wordlist = 'NA'):

    # tuple unpack hsl values
    hue, sat, lumibright, lumirand = hsl
    if len(lumirand) == 2:
        hue2, sat2 = hue, sat
        lumil, lumiu = lumirand
    else:
        hue2, sat2, lumil, lumiu = lumirand
    
    # Set custom colour. The other args are currently unused but could be utilised to affect the color scheme. e.g. hue = len(word) * basehue to use word length to affect hue
    def color_func(word, font_size, position,orientation,random_state=None, **kwargs):
        if word in wordlist:
            return(f"hsl({hue}, {sat}%, {lumibright}%)")
        else:
            return(f"hsl({hue2}, {sat2}%, {np.random.randint(lumil , lumiu)}%)")
        
    wordcloud = WordCloud(font_path = 'Lato-Regular.ttf', width=width, height=height, collocations=False, max_words = size, background_color = 'white', normalize_plurals = False).generate_from_frequencies(freqdict)

    # Display the generated image:
    plt.figure(figsize=(20,10))
    plt.imshow(wordcloud.recolor(color_func = color_func), interpolation='bilinear')
    plt.axis("off")
    plt.savefig(f'Clouds/Emotions/{dirname}/{filename}.png',bbox_inches='tight')
    plt.close()

In [114]:
# Creates the word frequency dictionary for one color in one region

def wordfreqcreator(colordict, color, col):
    # Access df based on color key
    v = colordict[color]
    # Initialise word frequencies dictionary
    wordfreq = {}
    # Loop through rows and save freq to dict
    for i in range(len(v.index)):
        # [i,0] is the emotion, [i,col] is the respective frequency depending on region
        wordfreq[v.iloc[i,0]] = v.iloc[i,col]

    # get top 5 words
    sorted_wordfreq = dict(sorted(wordfreq.items(), key=lambda x:x[1], reverse = True))
    top5 = list(sorted_wordfreq.keys())[0:5]
    
    return wordfreq, top5

In [116]:
# main code for emotions
for k, v in regiondirections.items():
    for k2, v2 in emotionsdirections.items():
        wordfrequecies, top5words = wordfreqcreator(colordict, k2, v)

        cloudgeneratoremotion(wordfrequecies, v2[0], k2, v2[1], v2[2], v2[3], k, top5words)

In [260]:
len(colordict['green'].index)

45

In [336]:
sorted_wordfreq = dict(sorted(regiondirections.items(), key=lambda x:x[1], reverse = True))

In [340]:
list(sorted_wordfreq.keys())[0:5]

['MOROCCO', 'US', 'MEXICO', 'BRAZIL', 'FRANCE']

In [30]:
colordict['brown']

,EMOTIONS,GLOBAL,APAC,EUROPE,LATAM,CHINA,INDONESIA,INDIA,UK,GERMANY,FRANCE,BRAZIL,MEXICO,US,MOROCCO
0,Elegance,669,646,609,992,937,220,781,323,669,253,957,1026,328,1192
1,Sophistication,677,815,376,1202,1378,294,773,423,894,93,1432,971,413,94
2,Confidence,652,1137,350,608,1069,556,1786,879,289,76,419,797,497,155
3,Power,757,867,704,817,1284,171,1148,688,304,96,998,636,517,1729
4,Success,335,363,379,187,261,239,591,211,516,499,275,100,370,292
5,Trust,681,609,735,775,770,435,623,500,762,525,1094,457,494,1153
6,Comfort,1104,1278,978,764,952,1372,1511,1516,1061,690,601,928,1760,645
7,Naturality,1700,1504,1798,1537,725,1301,2488,2571,1728,1151,1961,1112,2226,1741
8,Organic,1710,1455,1550,2243,1398,726,2241,1964,2271,687,2975,1512,2054,1276
9,Warmth,1842,1704,2057,1456,1083,2274,1756,1641,2223,1643,1101,1810,2166,2720
